# How to call functions with chat models

OpenAI Cookbook Colin Jarvis, Joe Palermo [Text Page Created on 13 June 2023]
(https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models)

Github Repository (where this file came from) =
[openai-cookbook/examples/How_to_call_functions_with_chat_models.ipynb](examples/How_to_call_functions_with_chat_models.ipynb)

-----------------------------------------------------------------------------

This notebook covers how to use the Chat Completions API in combination with external functions to extend the capabilities of GPT models.

`tools` is an optional parameter in the Chat Completion API which can be used to provide function specifications. The purpose of this is to enable models to generate function arguments which adhere to the provided specifications. Note that the API will not actually execute any function calls. It is up to developers to execute function calls using model outputs.

Within the `tools` parameter, if the `functions` parameter is provided then by default the model will decide when it is appropriate to use one of the functions. The API can be forced to use a specific function by setting the `tool_choice` parameter to `{"name": "<insert-function-name>"}`. The API can also be forced to not use any function by setting the `tool_choice` parameter to `"none"`. If a function is used, the output will contain `"finish_reason": "function_call"` in the response, as well as a `tool_choice` object that has the name of the function and the generated function arguments.

### Overview

This notebook contains the following 2 sections:

- **How to generate function arguments:** Specify a set of functions and use the API to generate function arguments.
- **How to call functions with model generated arguments:** Close the loop by actually executing functions with model generated arguments.

### by HG ###
I added extra notes as I thought appropriate.  I hope they help.

In [1]:
# Define the md() function to display markdown text
from IPython.display import display, Markdown
def md(s):
    display(Markdown(s))

# Define the mdc() function to display markdown text with a custom font size

def md_custom(text, size=12):
    return Markdown(f"<span style='font-size: {size}px;'>{text}</span>")

# Usage
# display(md_custom("This is some text with a larger font size.", size=18))
def mdc(s, size=14):
    display(md_custom(s, size))


## How to generate function arguments

In [2]:
%pip install scipy
%pip install tenacity
%pip install tiktoken
%pip install termcolor 
%pip install openai


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install scipy
%pip install tenacity
%pip install tiktoken
%pip install termcolor 
%pip install openai


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored  

GPT_MODEL = "gpt-3.5-turbo-0613"

from ute import init_openai
(Client, LLM)=init_openai(model=GPT_MODEL)

### Utilities

First let's define a few utilities for making calls to the Chat Completions API and for maintaining and keeping track of the conversation state.

### `chat_completion_request` ###
The following code snippet defines a Python function chat_completion_request decorated with the @retry decorator from a library likely similar to tenacity.

The decorator `@retry` is used to automatically retry the decorated function under specific conditions. The parameters for the retry behavior are defined using `wait=wait_random_exponential(multiplier=1, max=40)` and `stop=stop_after_attempt(3)`.

`wait=wait_random_exponential(multiplier=1, max=40)`: This means that the wait time between retries follows a random exponential pattern starting with a multiplier of 1 second and will not exceed 40 seconds. This strategy helps in handling cases like rate limits or temporary network issues by waiting for an exponentially increasing duration before each retry, with some randomness to prevent thundering herd problems.

`stop=stop_after_attempt(3)`: This specifies that the function should stop retrying after 3 attempts. If the function still fails after 3 attempts, it will not be retried further, and the exception handling code within the function will be executed.

Function `chat_completion_request`: This function takes several parameters: messages, tools, tool_choice, and model with `GPT_MODEL` as its default value. It is designed to make a request to a chat completion API (presumably an AI or machine learning model endpoint, such as one provided by OpenAI or a similar service) with the given parameters.

`Try-Except` Block: Inside the function, there's a try-except block intended to capture and handle any exceptions that occur during the execution of the API call. The API call is made using Client.chat.completions.create, which likely sends a request to a chat model with the specified parameters.

If the API call is successful, the response is returned from the function.
If an exception occurs, it is caught by the except block, which then prints an error message along with the exception details and returns the exception object.
The primary purpose of this function is to provide a robust way to interact with a chat completion API, handling potential errors and retrying the request automatically under certain conditions to increase the likelihood of success in cases of temporary issues.

In [5]:
from tenacity import retry, wait_random_exponential


@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = Client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


### `pretty_print_conversation` ###
The following function is designed to print messages from a conversation with color-coded roles for better readability and distinction between participants. It uses a role_to_color dictionary to map each role (system, user, assistant, and function) to a specific color (red, green, blue, and magenta, respectively).

In [6]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


### Basic concepts

Let's create some function specifications to interface with a hypothetical weather API. We'll pass these function specification to the Chat Completions API in order to generate function arguments that adhere to the specification.

In `tools` below, we include two function specifications.  The API will try to select the correct function based on the contents of our message.  If the message is not relevant any of these two functions, the API will answer it as answering a general chat query.

In [7]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },
]

If we prompt the model about the current weather, it will respond with some clarifying questions.

In [8]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What is the current weather?"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message


ChatCompletionMessage(content='Sure, could you please provide me with the location and the temperature unit you would like to use (celsius or fahrenheit)?', role='assistant', function_call=None, tool_calls=None)

Once we provide the missing information, it will generate the appropriate function arguments for us.

We provide the missing information in this instance by appending our location to the messages and calling `chat_completion` again.

Note that it will not call the function.  It will only identify which function to call and the calling format with the arguments.  It is up to our program to call the function and to produce the results.  We will see how to do it in this notebook further below.

In [9]:
messages.append({"role": "user", "content": "I'm in Glasgow, Scotland."})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
md(assistant_message.__str__())


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_uQwL3BQcfYvJqbV1nyligDpG', function=Function(arguments='{\n  "location": "Glasgow, Scotland",\n  "format": "celsius"\n}', name='get_current_weather'), type='function')])


By prompting it differently, we can get it to target the other function we've told it about.  This time we ask for a weather forecast in a given location.  But we do not specify the number of days.  We say "x days".

In [10]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in Glasgow, Scotland over the next x days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
md(assistant_message.__str__())


ChatCompletionMessage(content="Sure, could you please specify the value of 'x' for the number of days you would like to check the weather forecast for?", role='assistant', function_call=None, tool_calls=None)

Once again, the model is asking us for clarification because it doesn't have enough information yet. In this case it already knows the location for the forecast, but it needs to know how many days are required in the forecast.  So we send a new message text including this information:

In [11]:
messages.append({"role": "user", "content": "5 days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
md(chat_response.choices[0].message.__str__())


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_rjyIu0zHUOeraaoU9lNzPasP', function=Function(arguments='{\n  "location": "Glasgow, Scotland",\n  "format": "celsius",\n  "num_days": 5\n}', name='get_n_day_weather_forecast'), type='function')])

#### Forcing the use of specific functions or no function`

We can force the model to use a specific function.  This is done by using the `tool_choice` argument when calling `chat_completion_request`.

In the following, we provide `get_n_day_weather_forecast` as the name of the function in the `tool_choice` argument.

In [12]:
# in this cell we force the model to use get_n_day_weather_forecast
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice={"type": "function", "function": {"name": "get_n_day_weather_forecast"}}
)
md(chat_response.choices[0].message.__str__())

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_WBGzNbvpHx0XaqY2x22vDrPk', function=Function(arguments='{\n  "location": "Toronto, Canada",\n  "format": "celsius",\n  "num_days": 1\n}', name='get_n_day_weather_forecast'), type='function')])

If we do not force-feed the function, the GPT may still use the function depending on the text of our message but this is not guaranteed.  The following, for example, picks the right function but do not include the `num_days` in the call format, whereas the preceding cell did include `num_days`.  The only difference between the two is we force-fed the function `get_n_day_weather_forecast` in the preceding cell.

This implies to me that the function calls could be quite flaky and one needs to be careful.

In [13]:
# if we don't force the model to use get_n_day_weather_forecast it may not
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools
)
md(chat_response.choices[0].message.__str__())

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_mESN3aH3tdQhxqCSg6Pd7RDH', function=Function(arguments='{\n  "location": "Toronto, Canada",\n  "format": "celsius"\n}', name='get_current_weather'), type='function')])

### Parallel Function Calling

Newer models like gpt-4-1106-preview or gpt-3.5-turbo-1106 can call multiple functions in one turn.

In [14]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in San Francisco and Glasgow over the next 4 days"})
chat_response = chat_completion_request(
    messages, tools=tools, model='gpt-3.5-turbo-1106'
)

assistant_message = chat_response.choices[0].message.tool_calls
md(assistant_message.__str__())

[ChatCompletionMessageToolCall(id='call_dRgUOZfiuGwY9k8pgWNrEe3J', function=Function(arguments='{"location": "San Francisco, CA", "format": "celsius", "num_days": 4}', name='get_n_day_weather_forecast'), type='function'), ChatCompletionMessageToolCall(id='call_BceVFsnsrs6VLTkXT0J3JSsJ', function=Function(arguments='{"location": "Glasgow", "format": "celsius", "num_days": 4}', name='get_n_day_weather_forecast'), type='function')]

## How to call functions with model generated arguments

In our next example, we'll demonstrate how to execute functions whose inputs are model-generated, and use this to implement an agent that can answer questions for us about a database. For simplicity we'll use the [Chinook sample database](https://www.sqlitetutorial.net/sqlite-sample-database/). The Chinook database is a sample database. It represents a digital media store, including tables for artists, albums, media tracks, invoices, and customers.


The `chinook.db` is not included in `sqlite3`.  Download it from the [openai/openai-cookbook github folder](https://github.com/openai/openai-cookbook).

*Note:* SQL generation can be high-risk in a production environment since models are not perfectly reliable at generating correct SQL.

### Specifying a function to execute SQL queries

First let's define some helpful utility functions to extract data from a SQLite database.  `sqlite3` below is a built-in Python module that provides a lightweight disk-based database.


In [18]:
import sqlite3

conn = sqlite3.connect("data/Chinook.db")
print("Opened database successfully")

Opened database successfully


In [19]:
def get_table_names(conn):
    """Return a list of table names."""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names


def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names


def get_database_info(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts


Now can use these utility functions to extract a representation of the database schema.

In [21]:
database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

As before, we'll define a function specification for the function we'd like the API to generate arguments for. Notice that we are inserting the database schema into the function specification. This will be important for the model to know about.

In [22]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "ask_database",
            "description": "Use this function to answer user questions about music. Input should be a fully formed SQL query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": f"""
                                SQL query extracting info to answer the user's question.
                                SQL should be written using this database schema:
                                {database_schema_string}
                                The query should be returned in plain text, not in JSON.
                                """,
                    }
                },
                "required": ["query"],
            },
        }
    }
]

### Executing SQL queries

Now let's implement the function that will actually excute queries against the database.

In [23]:
def ask_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

def execute_function_call(message):
    if message.tool_calls[0].function.name == "ask_database":
        query = json.loads(message.tool_calls[0].function.arguments)["query"]
        results = ask_database(conn, query)
    else:
        results = f"Error: function {message.tool_calls[0].function.name} does not exist"
    return results

The following is our conservation with the ChatCompletions GPT.  The Instructions for the GPT are pushed into messages the first.  Those instructions are:
* `messages[0]`: "Answer user questions by generating SQL queries against the Chinook Music Database."

We then want to find out the top 5 artists according to this database.  So we pose the following query:
* `messages[1]`:"Hi, who are the top 5 artists by number of tracks?"

When we call `chat_completion_request(messages, tools)`, the GPT pushes the message to the top of `messages`, which is:
* `messages[2]` : "Function(arguments='{\n  "query": "SELECT artist.Name, COUNT(track.TrackId) AS num_tracks FROM artist JOIN album ON artist.ArtistId = album.ArtistId JOIN track ON album.AlbumId = track.AlbumId GROUP BY artist.ArtistId ORDER BY num_tracks DESC LIMIT 5"\n}', name='ask_database')"

We should recognise the significance of what is happening here:
* The GPT recognises that our question is of the type that can be answered by querying the database. 
* It checks its `tools` and finds out that we provided the GPT with a function `ask_database` that is called in SQL format using the database scheme `database_schema_string`.  This is a string defined earlier in this notebook.
* The GPT uses the schema to generate a query to the database.

So all we have to do is pass that query to the database.  This we do by calling the function named in GPT answer, which is `ask_database`.  

### The ordering in the list `messages` ###
Note that the ordering in `messages` here is different from the order in the messages corresponding to a Thread in Assistants API.  In Assistants API, the messages are ordered in reverse. The last message is the initial question. The second last message is the OpenAI answer to this question. See the notebook `playground.ipynb` in this `probot` repository for an example.

Here, in ChatCompletionsGPT, it is exactly the opposite.  The first message always stays the first and the new messages are added to the end of the list.  I find this more logical.  I am not sure why they do it differently in Assistants API.


In [52]:
messages = []
messages.append({"role": "system", "content": "Answer user questions by generating SQL queries against the Chinook Music Database."})
messages.append({"role": "user", "content": "Hi, who are the top 5 artists by number of tracks?"})
chat_response = chat_completion_request(messages, tools)
assistant_message = chat_response.choices[0].message
assistant_message.content = str(assistant_message.tool_calls[0].function)
messages.append({"role": assistant_message.role, "content": assistant_message.content})
if assistant_message.tool_calls:
    results = execute_function_call(assistant_message)
    messages.append({"role": "function", "tool_call_id": assistant_message.tool_calls[0].id,
                     "name": assistant_message.tool_calls[0].function.name, "content": results})
pretty_print_conversation(messages)

system: Answer user questions by generating SQL queries against the Chinook Music Database.

user: Hi, who are the top 5 artists by number of tracks?

assistant: Function(arguments='{\n  "query": "SELECT a.Name AS Artist, COUNT(t.TrackId) AS TrackCount FROM Artist a INNER JOIN Album al ON a.ArtistId = al.ArtistId INNER JOIN Track t ON al.AlbumId = t.AlbumId GROUP BY a.ArtistId ORDER BY TrackCount DESC LIMIT 5"\n}', name='ask_database')

function (ask_database): [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]



In the following cell, we place one more query to the ChatCompletions GPT.  This question is placed at the ends of the messages list,
```
messages.append({"role": "user", "content": "What is the name of the album with the most tracks?"})
```
 and the GPT is called again as before.

In [56]:
messages.append({"role": "user", "content": "What is the name of the album with the most tracks?"})
chat_response = chat_completion_request(messages, tools)
assistant_message = chat_response.choices[0].message
assistant_message.content = str(assistant_message.tool_calls[0].function)
messages.append({"role": assistant_message.role, "content": assistant_message.content})
if assistant_message.tool_calls:
    results = execute_function_call(assistant_message)
    messages.append({"role": "function", "tool_call_id": assistant_message.tool_calls[0].id, "name": assistant_message.tool_calls[0].function.name, "content": results})
pretty_print_conversation(messages)

system: Answer user questions by generating SQL queries against the Chinook Music Database.

user: Hi, who are the top 5 artists by number of tracks?

assistant: Function(arguments='{\n  "query": "SELECT a.Name AS Artist, COUNT(t.TrackId) AS TrackCount FROM Artist a INNER JOIN Album al ON a.ArtistId = al.ArtistId INNER JOIN Track t ON al.AlbumId = t.AlbumId GROUP BY a.ArtistId ORDER BY TrackCount DESC LIMIT 5"\n}', name='ask_database')

function (ask_database): [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]

user: What is the name of the album with the most tracks?

assistant: Function(arguments='{\n  "query": "SELECT a.Title AS Album, COUNT(t.TrackId) AS TrackCount FROM Album a INNER JOIN Track t ON a.AlbumId = t.AlbumId GROUP BY a.AlbumId ORDER BY TrackCount DESC LIMIT 1"\n}', name='ask_database')

function (ask_database): [('Greatest Hits', 57)]

